# Initial exploration of each data file
Pandas makes importing data from files easy. But sometimes the file contents are poorly formatted or can hold hidden surprises. Make sure that the data - and data types - are what you expect them to be before starting your analysis.

In [1]:
import pandas as pd
import numpy as np
import os
from os.path import join

cwd = os.getcwd()
data_path = join(cwd, '..', '..', 'data')

### Don't write absolute paths
An absolute path is something like `/Users/Home/Documents/GitHub/python-data-analysis-class/data/epa_emissions_2016.txt`. 

In [2]:
# Paths to each of the data files (epa emissions, eia capacity by generator, and eia generation)

epa_path = join(data_path, 'epa_epa_2016.txt')
cap_path = join(data_path, '3_1_Generator_Y2016.xlsx')
gen_path = join(data_path, 'EIA923_Schedules_2_3_4_5_M_10_2016.xlsx')

## Load EPA epa data
Lets load the file and see what needs to be done to make sure the data is in good shape and accessible.

In [3]:
epa = pd.read_csv(epa_path)

It looks like the header column is not well aligned with the data. States are showing up as the index and the first column is labeled **State**.

In [7]:
epa.head()

,State,Facility Name,Facility ID (ORISPL),Month,Year,Gross Load (MW-h),SO2 (tons),NOx (tons),CO2 (short tons),Heat Input (MMBtu)
AL,AMEA Sylacauga Plant,56018,1,2016,4534.0,0.014,2.229,3101.8,52585.8,NaN
AL,AMEA Sylacauga Plant,56018,2,2016,792.0,0.002,0.361,542.0,9186.0,NaN
AL,AMEA Sylacauga Plant,56018,3,2016,1498.0,0.005,0.677,1024.2,17365.5,NaN
AL,AMEA Sylacauga Plant,56018,4,2016,1405.0,0.005,0.586,884.1,14987.3,NaN
AL,AMEA Sylacauga Plant,56018,5,2016,1791.0,0.006,0.756,1145.1,19412.8,NaN


In [9]:
with open(epa_path) as f:
    f.read()

In [27]:
with open(epa_path) as f:
    head = [next(f) for x in range(5)]

for line in head:
    print(line) 

State, Facility Name, Facility ID (ORISPL), Month, Year, Gross Load (MW-h), SO2 (tons), NOx (tons), CO2 (short tons), Heat Input (MMBtu)

"AL","AMEA Sylacauga Plant","56018","1","2016","4534","0.014","2.229","3101.8","52585.8",

"AL","AMEA Sylacauga Plant","56018","2","2016","792","0.002","0.361","542","9186",

"AL","AMEA Sylacauga Plant","56018","3","2016","1498","0.005","0.677","1024.2","17365.5",

"AL","AMEA Sylacauga Plant","56018","4","2016","1405","0.005","0.586","884.1","14987.3",



In [28]:
epa = pd.read_csv(epa_path, index_col=False)

In [29]:
epa.head()

,State,Facility Name,Facility ID (ORISPL),Month,Year,Gross Load (MW-h),SO2 (tons),NOx (tons),CO2 (short tons),Heat Input (MMBtu)
0,AL,AMEA Sylacauga Plant,56018,1,2016,4534.0,0.014,2.229,3101.8,52585.8
1,AL,AMEA Sylacauga Plant,56018,2,2016,792.0,0.002,0.361,542.0,9186.0
2,AL,AMEA Sylacauga Plant,56018,3,2016,1498.0,0.005,0.677,1024.2,17365.5
3,AL,AMEA Sylacauga Plant,56018,4,2016,1405.0,0.005,0.586,884.1,14987.3
4,AL,AMEA Sylacauga Plant,56018,5,2016,1791.0,0.006,0.756,1145.1,19412.8


In [30]:
epa.tail()

,State,Facility Name,Facility ID (ORISPL),Month,Year,Gross Load (MW-h),SO2 (tons),NOx (tons),CO2 (short tons),Heat Input (MMBtu)
14476,WY,Wyodak,6101,8,2016,279246.0,233.875,347.809,321484.0,3065249.8
14477,WY,Wyodak,6101,9,2016,267691.0,228.710,334.644,309080.6,2946991.1
14478,WY,Wyodak,6101,10,2016,253110.0,211.495,314.882,290879.0,2773451.5
14479,WY,Wyodak,6101,11,2016,249476.0,207.293,306.374,283543.1,2703507.4
14480,WY,Wyodak,6101,12,2016,252072.0,207.199,304.010,282007.9,2688870.4


### Access parts of the dataframe

Look at the column names

In [32]:
epa.columns

Index(['State', ' Facility Name', ' Facility ID (ORISPL)', ' Month', ' Year',
       ' Gross Load (MW-h)', ' SO2 (tons)', ' NOx (tons)', ' CO2 (short tons)',
       ' Heat Input (MMBtu)'],
      dtype='object')

Notice that most of the columns have a leading space? We need to strip out those leading spaces and it might be nice to do some extra formatting.

In [9]:
epa.columns.str.strip()

Index(['State', 'Facility Name', 'Facility ID (ORISPL)', 'Unit ID',
       'Associated Stacks', 'Month', 'Year', 'Program(s)', 'Operating Time',
       'Gross Load (MW-h)', 'Steam Load (1000lb)', 'SO2 (tons)',
       'Avg. NOx Rate (lb/MMBtu)', 'NOx (tons)', 'CO2 (short tons)',
       'Heat Input (MMBtu)', 'EPA Region', 'NERC Region', 'Unit Type',
       'Fuel Type (Primary)'],
      dtype='object')

In [36]:
epa.columns = epa.columns.str.strip()

In [37]:
epa.columns = (epa.columns.str.lower()
                  .str.replace(' ', '_')
                  .str.replace('.', '')
                  .str.replace('-', '')
                  .str.replace('(', '')
                  .str.replace(')', ''))

### Data types of each column
Numeric columns will either be `int` or `float`. If a column is of type `object` it is either all strings or a mix of types. Watch out for columns that should be numeric but should up as `object`.

In [38]:
epa.dtypes

state                  object
facility_name          object
facility_id_orispl      int64
month                   int64
year                    int64
gross_load_mwh        float64
so2_tons              float64
nox_tons              float64
co2_short_tons        float64
heat_input_mmbtu      float64
dtype: object

## Basic statistics of the data

In [44]:
epa.describe()

,facility_id_orispl,month,year,gross_load_mwh,so2_tons,nox_tons,co2_short_tons,heat_input_mmbtu
count,14481.000000,14481.000000,14481.0,1.226200e+04,12465.000000,12700.000000,1.202200e+04,1.272200e+04
mean,22422.507769,6.490988,2016.0,1.974383e+05,117.769365,91.650835,1.582862e+05,1.743909e+06
std,24871.051651,3.442153,0.0,2.926614e+05,369.451131,214.962978,2.678391e+05,2.650419e+06
min,3.000000,1.000000,2016.0,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00
25%,2399.000000,4.000000,2016.0,4.474020e+03,0.028000,1.553750,4.555913e+03,5.153396e+04
50%,7145.000000,6.000000,2016.0,5.686740e+04,0.356000,9.534000,4.536591e+04,5.493512e+05
75%,55238.000000,9.000000,2016.0,2.890370e+05,16.868000,55.851500,1.799689e+05,2.454411e+06
max,70454.000000,12.000000,2016.0,2.093063e+06,5165.046000,2394.967000,2.341848e+06,2.232883e+07


Index into a dataframe using `.loc` or `.iloc` with square brackets and row,column notation

In [41]:
epa.iloc[0:5, :3]

,state,facility_name,facility_id_orispl
0,AL,AMEA Sylacauga Plant,56018
1,AL,AMEA Sylacauga Plant,56018
2,AL,AMEA Sylacauga Plant,56018
3,AL,AMEA Sylacauga Plant,56018
4,AL,AMEA Sylacauga Plant,56018


## Load capacity data

In [47]:
capacity = pd.read_excel(cap_path, sheet_name='Operable')

/Users/Home/anaconda/envs/pydata-issst/lib/python3.6/site-packages/pandas/io/excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


In [48]:
capacity.head()

,"2016 Form EIA-860 Data - Schedule 3, 'Generator Data' (Operable Units Only)",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72
0,Utility ID,Utility Name,Plant Code,Plant Name,State,County,Generator ID,Technology,Prime Mover,Unit Code,...,Planned Energy Source 1,Planned New Nameplate Capacity (MW),Planned Repower Month,Planned Repower Year,Other Planned Modifications?,Other Modifications Month,Other Modifications Year,Multiple Fuels?,Cofire Fuels?,Switch Between Oil and Natural Gas?
1,195,Alabama Power Co,2,Bankhead Dam,AL,Tuscaloosa,1,Conventional Hydroelectric,HY,NaN,...,NaN,,,,NaN,,,N,NaN,NaN
2,195,Alabama Power Co,3,Barry,AL,Mobile,1,Natural Gas Steam Turbine,ST,NaN,...,NaN,,,,NaN,,,N,NaN,NaN
3,195,Alabama Power Co,3,Barry,AL,Mobile,2,Natural Gas Steam Turbine,ST,NaN,...,NaN,,,,NaN,,,N,NaN,NaN
4,195,Alabama Power Co,3,Barry,AL,Mobile,4,Conventional Steam Coal,ST,NaN,...,NaN,,,,NaN,,,N,NaN,NaN


Looks like the first row isn't the column names and the last row is a footnote

In [50]:
capacity.tail()

,Utility ID,Utility Name,Plant Code,Plant Name,State,County,Generator ID,Technology,Prime Mover,Unit Code,...,Planned Energy Source 1,Planned New Nameplate Capacity (MW),Planned Repower Month,Planned Repower Year,Other Planned Modifications?,Other Modifications Month,Other Modifications Year,Multiple Fuels?,Cofire Fuels?,Switch Between Oil and Natural Gas?
20720,61053,Mount Sinai Hospital,61416.0,Mount Sinai Hospital,NY,New York,GP8,Petroleum Liquids,IC,NaN,...,NaN,,,,NaN,,,N,NaN,NaN
20721,61053,Mount Sinai Hospital,61416.0,Mount Sinai Hospital,NY,New York,GP9,Petroleum Liquids,IC,NaN,...,NaN,,,,NaN,,,N,NaN,NaN
20722,61053,Mount Sinai Hospital,61416.0,Mount Sinai Hospital,NY,New York,HES13,Petroleum Liquids,IC,NaN,...,NaN,,,,NaN,,,N,NaN,NaN
20723,61053,Mount Sinai Hospital,61416.0,Mount Sinai Hospital,NY,New York,HES14,Petroleum Liquids,IC,NaN,...,NaN,,,,NaN,,,N,NaN,NaN
20724,NOTE: Information on planned capacity changes ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
capacity = pd.read_excel(cap_path, sheet_name='Operable', header=1, skipfooter=1)

In [52]:
capacity.head()

,Utility ID,Utility Name,Plant Code,Plant Name,State,County,Generator ID,Technology,Prime Mover,Unit Code,...,Planned Energy Source 1,Planned New Nameplate Capacity (MW),Planned Repower Month,Planned Repower Year,Other Planned Modifications?,Other Modifications Month,Other Modifications Year,Multiple Fuels?,Cofire Fuels?,Switch Between Oil and Natural Gas?
0,195,Alabama Power Co,2,Bankhead Dam,AL,Tuscaloosa,1,Conventional Hydroelectric,HY,NaN,...,NaN,,,,NaN,,,N,NaN,NaN
1,195,Alabama Power Co,3,Barry,AL,Mobile,1,Natural Gas Steam Turbine,ST,NaN,...,NaN,,,,NaN,,,N,NaN,NaN
2,195,Alabama Power Co,3,Barry,AL,Mobile,2,Natural Gas Steam Turbine,ST,NaN,...,NaN,,,,NaN,,,N,NaN,NaN
3,195,Alabama Power Co,3,Barry,AL,Mobile,4,Conventional Steam Coal,ST,NaN,...,NaN,,,,NaN,,,N,NaN,NaN
4,195,Alabama Power Co,3,Barry,AL,Mobile,5,Conventional Steam Coal,ST,NaN,...,NaN,,,,NaN,,,N,NaN,NaN


### Check the column names


In [53]:
capacity.columns

Index(['Utility ID', 'Utility Name', 'Plant Code', 'Plant Name', 'State',
       'County', 'Generator ID', 'Technology', 'Prime Mover', 'Unit Code',
       'Ownership', 'Duct Burners',
       'Can Bypass Heat Recovery Steam Generator?',
       'RTO/ISO LMP Node Designation',
       'RTO/ISO Location Designation for Reporting Wholesale Sales Data to FERC',
       'Nameplate Capacity (MW)', 'Nameplate Power Factor',
       'Summer Capacity (MW)', 'Winter Capacity (MW)', 'Minimum Load (MW)',
       'Uprate or Derate Completed During Year',
       'Month Uprate or Derate Completed', 'Year Uprate or Derate Completed',
       'Status', 'Synchronized to Transmission Grid', 'Operating Month',
       'Operating Year', 'Planned Retirement Month', 'Planned Retirement Year',
       'Associated with Combined Heat and Power System', 'Sector Name',
       'Sector', 'Topping or Bottoming', 'Energy Source 1', 'Energy Source 2',
       'Energy Source 3', 'Energy Source 4', 'Energy Source 5',
       'Ene

In [54]:
capacity.dtypes

Utility ID                                                                   int64
Utility Name                                                                object
Plant Code                                                                   int64
Plant Name                                                                  object
State                                                                       object
County                                                                      object
Generator ID                                                                object
Technology                                                                  object
Prime Mover                                                                 object
Unit Code                                                                   object
Ownership                                                                   object
Duct Burners                                                                object
Can 

In [71]:
capacity.columns = [name.strip() for name in capacity.columns]
capacity.columns

Index([u'Entity ID', u'Entity Name', u'Plant ID', u'Plant Name', u'Sector',
       u'Plant State', u'Generator ID', u'Nameplate Capacity (MW)',
       u'Net Summer Capacity (MW)', u'Technology', u'Energy Source Code',
       u'Prime Mover Code', u'Operating Month', u'Operating Year',
       u'Planned Retirement Month', u'Planned Retirement Year', u'Status',
       u'Planned Derate Year', u'Planned Derate Month',
       u'Planned Derate of Summer Capacity (MW)', u'Planned Uprate Year',
       u'Planned Uprate Month', u'Planned Uprate of Summer Capacity (MW)',
       u'County', u'Latitude', u'Longitude'],
      dtype='object')

## Boolean filtering

In [43]:
PA_cap = capacity.loc[capacity['Plant State']=='PA',:]
PA_cap

,Entity ID,Entity Name,Plant ID,Plant Name,Sector,Plant State,Generator ID,Nameplate Capacity (MW),Net Summer Capacity (MW),Technology,...,Status,Planned Derate Year,Planned Derate Month,Planned Derate of Summer Capacity (MW),Planned Uprate Year,Planned Uprate Month,Planned Uprate of Summer Capacity (MW),County,Latitude,Longitude
5223,14165,NRG Power Midwest LP,3096,Brunot Island,IPP Non-CHP,PA,1A,25.5,15,Petroleum Liquids,...,(OP) Operating,,,,,,,Allegheny,40.4649,-80.0438
5224,14165,NRG Power Midwest LP,3096,Brunot Island,IPP Non-CHP,PA,2A,65.3,46,Natural Gas Fired Combined Cycle,...,(OP) Operating,,,,,,,Allegheny,40.4649,-80.0438
5225,14165,NRG Power Midwest LP,3096,Brunot Island,IPP Non-CHP,PA,2B,65.3,48,Natural Gas Fired Combined Cycle,...,(OP) Operating,,,,,,,Allegheny,40.4649,-80.0438
5226,14165,NRG Power Midwest LP,3096,Brunot Island,IPP Non-CHP,PA,3,65.3,49,Natural Gas Fired Combined Cycle,...,(OP) Operating,,,,,,,Allegheny,40.4649,-80.0438
5227,14165,NRG Power Midwest LP,3096,Brunot Island,IPP Non-CHP,PA,ST4,144.0,101,Natural Gas Fired Combined Cycle,...,(OP) Operating,,,,,,,Allegheny,40.4649,-80.0438
5228,17235,NRG REMA LLC,3109,Hamilton (PA),IPP Non-CHP,PA,1,19.6,18,Petroleum Liquids,...,(OP) Operating,,,,,,,Adams,39.9087,-76.9885
5229,17235,NRG REMA LLC,3110,Hunterstown,IPP Non-CHP,PA,1,20.0,18,Petroleum Liquids,...,(OP) Operating,,,,,,,Adams,39.8662,-77.1648
5230,17235,NRG REMA LLC,3110,Hunterstown,IPP Non-CHP,PA,2,20.0,17,Petroleum Liquids,...,(OP) Operating,,,,,,,Adams,39.8662,-77.1648
5231,17235,NRG REMA LLC,3110,Hunterstown,IPP Non-CHP,PA,3,20.0,18,Petroleum Liquids,...,(OP) Operating,,,,,,,Adams,39.8662,-77.1648
5232,17235,NRG REMA LLC,3111,Mountain,IPP Non-CHP,PA,1,27.0,18,Petroleum Liquids,...,(OP) Operating,,,,,,,Cumberland,40.1229,-77.1723


In [45]:
PA_NGCC_cap = capacity.loc[(capacity['Plant State']=='PA') &
             (capacity['Technology']=='Natural Gas Fired Combined Cycle'),:]
PA_NGCC_cap

,Entity ID,Entity Name,Plant ID,Plant Name,Sector,Plant State,Generator ID,Nameplate Capacity (MW),Net Summer Capacity (MW),Technology,...,Status,Planned Derate Year,Planned Derate Month,Planned Derate of Summer Capacity (MW),Planned Uprate Year,Planned Uprate Month,Planned Uprate of Summer Capacity (MW),County,Latitude,Longitude
5224,14165,NRG Power Midwest LP,3096,Brunot Island,IPP Non-CHP,PA,2A,65.3,46,Natural Gas Fired Combined Cycle,...,(OP) Operating,,,,,,,Allegheny,40.4649,-80.0438
5225,14165,NRG Power Midwest LP,3096,Brunot Island,IPP Non-CHP,PA,2B,65.3,48,Natural Gas Fired Combined Cycle,...,(OP) Operating,,,,,,,Allegheny,40.4649,-80.0438
5226,14165,NRG Power Midwest LP,3096,Brunot Island,IPP Non-CHP,PA,3,65.3,49,Natural Gas Fired Combined Cycle,...,(OP) Operating,,,,,,,Allegheny,40.4649,-80.0438
5227,14165,NRG Power Midwest LP,3096,Brunot Island,IPP Non-CHP,PA,ST4,144.0,101,Natural Gas Fired Combined Cycle,...,(OP) Operating,,,,,,,Allegheny,40.4649,-80.0438
5381,19391,UGI Development Co,3176,Hunlock Power Station,IPP Non-CHP,PA,3,49.9,30.1,Natural Gas Fired Combined Cycle,...,(OP) Operating,,,,,,,Luzerne,41.2006,-76.07
5382,19391,UGI Development Co,3176,Hunlock Power Station,IPP Non-CHP,PA,5,48.0,48.7,Natural Gas Fired Combined Cycle,...,(OP) Operating,,,,,,,Luzerne,41.2006,-76.07
5383,19391,UGI Development Co,3176,Hunlock Power Station,IPP Non-CHP,PA,6,48.0,48.4,Natural Gas Fired Combined Cycle,...,(OP) Operating,,,,,,,Luzerne,41.2006,-76.07
11869,2468,Bucknell University,54333,Bucknell University,Commercial CHP,PA,G001,4.7,4.3,Natural Gas Fired Combined Cycle,...,(OP) Operating,,,,,,,Union,40.955,-76.8788
11870,2468,Bucknell University,54333,Bucknell University,Commercial CHP,PA,G502,1.2,0.5,Natural Gas Fired Combined Cycle,...,(OP) Operating,,,,,,,Union,40.955,-76.8788
12320,56516,"Morris Energy Operations Company, LLC",54693,York Generation Company LLC,IPP Non-CHP,PA,GT#1,8.3,46.2,Natural Gas Fired Combined Cycle,...,(OP) Operating,,,,,,,York,39.9856,-76.6762


### Repeat `groupby` and `sum` to get capacity of facilities

In [72]:
cols = ['Plant ID', 'Nameplate Capacity (MW)']
facility_cap = capacity.loc[:,cols].groupby('Plant ID').sum()
facility_cap

,Nameplate Capacity (MW)
Plant ID,
2,53.9
3,2569.5
4,225.0
7,138.0
8,1166.7
9,80.5
10,1288.4
11,72.9
12,46.9


# Load generation data

In [4]:
generation = pd.read_excel(eia923_path, header=5, na_values='.')

In [5]:
generation.columns = ((generation.columns.str.strip()
                             .str.lower()
                             .str.replace('\n', ' ')
                             .str.replace(' ', '_')
                             .str.replace('-', '')
                             .str.replace('(', '')
                             .str.replace(')', '')))

Something weird is going on here. I know there are lots of rows with numeric data that are missing from this `describe` table.

In [8]:
value_cols = [col for col in generation.columns if 'netgen' in col]

In [14]:
df = pd.melt(generation, id_vars=['plant_id'],
        value_vars=value_cols, value_name='net_gen',
        var_name='month').groupby(['plant_id', 'month'], as_index=False).sum()

In [15]:
df['month'] = df.month.str.replace('netgen_', '')

In [16]:
df.head()

,plant_id,month,net_gen
0,3,april,788708.000
1,3,august,1291524.003
2,3,december,0.000
3,3,february,977043.999
4,3,january,1123279.005


In [6]:
generation.describe()

,plant_id,nuclear_unit_id,operator_id,reserved,naics_code,eia_sector_number,reserved.1,reserved.2,quantity_january,quantity_february,...,netgen_september,netgen_october,netgen_november,netgen_december,total_fuel_consumption_quantity,electric_fuel_consumption_quantity,total_fuel_consumption_mmbtu,elec_fuel_consumption_mmbtu,net_generation_megawatthours,year
count,6846.000000,100.000000,6846.000000,0.0,6846.000000,6846.000000,0.0,0.0,6.638000e+03,6.640000e+03,...,6.665000e+03,6.689000e+03,0.0,0.0,6.846000e+03,6.846000e+03,6.846000e+03,6.846000e+03,6.846000e+03,6846.0
mean,47395.247590,1.590000,45494.412650,NaN,38365.911043,2.550394,NaN,NaN,1.678129e+05,1.512982e+05,...,5.276704e+04,4.676155e+04,NaN,NaN,1.735993e+06,1.523509e+06,5.046923e+06,4.736994e+06,5.018362e+05,2016.0
std,39659.073362,0.697687,38449.729044,NaN,68095.379303,2.051354,NaN,NaN,6.752423e+05,6.310197e+05,...,1.475519e+05,1.382842e+05,NaN,NaN,6.397009e+06,5.648343e+06,1.423608e+07,1.412352e+07,1.379263e+06,0.0
min,3.000000,1.000000,21.000000,NaN,22.000000,1.000000,NaN,NaN,0.000000e+00,0.000000e+00,...,-8.735000e+04,-7.369400e+04,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-7.028300e+05,2016.0
25%,6001.250000,1.000000,12492.000000,NaN,22.000000,1.000000,NaN,NaN,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,6.762500e+02,4.747500e+02,1.809423e+02,2016.0
50%,55061.000000,1.000000,22129.000000,NaN,22.000000,2.000000,NaN,NaN,3.050000e+01,7.000000e+00,...,3.021806e+03,2.042451e+03,NaN,NaN,2.378500e+03,1.470500e+03,3.173170e+05,2.088960e+05,3.127657e+04,2016.0
75%,99999.000000,2.000000,99999.000000,NaN,99999.000000,3.000000,NaN,NaN,2.985825e+04,2.613150e+04,...,2.804500e+04,2.460200e+04,NaN,NaN,4.469722e+05,2.983672e+05,2.892082e+06,2.288356e+06,2.921652e+05,2016.0
max,99999.000000,4.000000,99999.000000,NaN,562213.000000,7.000000,NaN,NaN,1.553549e+07,1.452921e+07,...,1.851736e+06,1.721639e+06,NaN,NaN,1.533434e+08,1.208176e+08,1.505038e+08,1.505038e+08,1.544404e+07,2016.0


In [74]:
generation.head()

,Plant Id,Combined Heat And Power Plant,Nuclear Unit Id,Plant Name,Operator Name,Operator Id,Plant State,Census Region,NERC Region,Reserved,...,Netgen September,Netgen October,Netgen November,Netgen December,Total Fuel Consumption Quantity,Electric Fuel Consumption Quantity,Total Fuel Consumption MMBtu,Elec Fuel Consumption MMBtu,Net Generation (Megawatthours),YEAR
0,3,N,NaN,Barry,Alabama Power Co,195,AL,ESC,SERC,NaN,...,251839,219263,.,.,4782719,4782719,4877614,4877614,2386339.000,2016
1,3,N,NaN,Barry,Alabama Power Co,195,AL,ESC,SERC,NaN,...,463646,401506,.,.,42413079,42413079,43208127,43208127,4486082.000,2016
2,3,N,NaN,Barry,Alabama Power Co,195,AL,ESC,SERC,NaN,...,473794,286736,.,.,1773704,1773704,37426485,37426485,3706973.300,2016
3,3,N,NaN,Barry,Alabama Power Co,195,AL,ESC,SERC,NaN,...,11246.3,14387.7,.,.,857741,857741,874585,874585,86059.704,2016
4,4,N,NaN,Walter Bouldin Dam,Alabama Power Co,195,AL,ESC,SERC,NaN,...,3380,884,.,.,0,0,4239930,4239930,454977.000,2016


Turns out that there are lots of dots (.) where it is no value. I'm going to replace these with zeros.

In [75]:
generation.tail()

,Plant Id,Combined Heat And Power Plant,Nuclear Unit Id,Plant Name,Operator Name,Operator Id,Plant State,Census Region,NERC Region,Reserved,...,Netgen September,Netgen October,Netgen November,Netgen December,Total Fuel Consumption Quantity,Electric Fuel Consumption Quantity,Total Fuel Consumption MMBtu,Elec Fuel Consumption MMBtu,Net Generation (Megawatthours),YEAR
6841,99999,Y,NaN,State-Fuel Level Increment,State-Fuel Level Increment,99999,WY,MTN,NaN,NaN,...,20.03,20.073,.,.,2471,197,14391,1144,225.055,2016
6842,99999,Y,NaN,State-Fuel Level Increment,State-Fuel Level Increment,99999,WY,MTN,NaN,NaN,...,8371.86,6890.44,.,.,5750996,817170,5662749,804426,84996.866,2016
6843,99999,Y,NaN,State-Fuel Level Increment,State-Fuel Level Increment,99999,WY,MTN,NaN,NaN,...,282.565,112.177,.,.,7449697,152028,1362700,27810,3212.683,2016
6844,99999,Y,NaN,State-Fuel Level Increment,State-Fuel Level Increment,99999,WY,MTN,NaN,NaN,...,0,0,.,.,0,0,0,0,0.000,2016
6845,99999,Y,NaN,State-Fuel Level Increment,State-Fuel Level Increment,99999,WY,MTN,NaN,NaN,...,10293,7668.97,.,.,406340,92966,7110741,1626327,133198.240,2016


In [110]:
generation.replace('.', 0, inplace=True)

There are line breaks in the middle of column names. I don't see any breaks or spaces at the beginning or end of names, but will still `strip` just to be safe.

In [76]:
generation.columns

Index([u'Plant Id', u'Combined Heat And\nPower Plant', u'Nuclear Unit Id',
       u'Plant Name', u'Operator Name', u'Operator Id', u'Plant State',
       u'Census Region', u'NERC Region', u'Reserved', u'NAICS Code',
       u'EIA Sector Number', u'Sector Name', u'Reported\nPrime Mover',
       u'Reported\nFuel Type Code', u'AER\nFuel Type Code', u'Reserved.1',
       u'Reserved.2', u'Physical\nUnit Label', u'Quantity\nJanuary',
       u'Quantity\nFebruary', u'Quantity\nMarch', u'Quantity\nApril',
       u'Quantity\nMay', u'Quantity\nJune', u'Quantity\nJuly',
       u'Quantity\nAugust', u'Quantity\nSeptember', u'Quantity\nOctober',
       u'Quantity\nNovember', u'Quantity\nDecember', u'Elec_Quantity\nJanuary',
       u'Elec_Quantity\nFebruary', u'Elec_Quantity\nMarch',
       u'Elec_Quantity\nApril', u'Elec_Quantity\nMay', u'Elec_Quantity\nJune',
       u'Elec_Quantity\nJuly', u'Elec_Quantity\nAugust',
       u'Elec_Quantity\nSeptember', u'Elec_Quantity\nOctober',
       u'Elec_Quantity\

In [79]:
generation.columns = [name.strip().replace('\n', ' ') for name in generation.columns]
generation.columns

Index([u'Plant Id', u'Combined Heat And Power Plant', u'Nuclear Unit Id',
       u'Plant Name', u'Operator Name', u'Operator Id', u'Plant State',
       u'Census Region', u'NERC Region', u'Reserved', u'NAICS Code',
       u'EIA Sector Number', u'Sector Name', u'Reported Prime Mover',
       u'Reported Fuel Type Code', u'AER Fuel Type Code', u'Reserved.1',
       u'Reserved.2', u'Physical Unit Label', u'Quantity January',
       u'Quantity February', u'Quantity March', u'Quantity April',
       u'Quantity May', u'Quantity June', u'Quantity July', u'Quantity August',
       u'Quantity September', u'Quantity October', u'Quantity November',
       u'Quantity December', u'Elec_Quantity January',
       u'Elec_Quantity February', u'Elec_Quantity March',
       u'Elec_Quantity April', u'Elec_Quantity May', u'Elec_Quantity June',
       u'Elec_Quantity July', u'Elec_Quantity August',
       u'Elec_Quantity September', u'Elec_Quantity October',
       u'Elec_Quantity November', u'Elec_Quantity 

## Stack data by month rather than having multiple columns
Not sure if I'll have time for this section

I'm lazy and want to get a list of month names without typing them all

In [82]:
# could have done this as a list comprehension, but it would have been harder to read
months = []
for name in generation.columns:
    if 'Netgen' in name:
        month = name.split()[-1]
        months.append(month)
months

[u'January',
 u'February',
 u'March',
 u'April',
 u'May',
 u'June',
 u'July',
 u'August',
 u'September',
 u'October',
 u'November',
 u'December']

In [90]:
id_cols = ['Plant Id', 'Plant State', 'NERC Region', 'AER Fuel Type Code']
monthly_cols = []
def find_col_names(cols):
    for col in cols:
        if 'January' in col:
            monthly_cols.append(col.split()[0])

find_col_names(generation.columns)
id_cols + monthly_cols

['Plant Id',
 'Plant State',
 'NERC Region',
 'AER Fuel Type Code',
 u'Quantity',
 u'Elec_Quantity',
 u'MMBtuPer_Unit',
 u'Tot_MMBtu',
 u'Elec_MMBtu',
 u'Netgen']

In [91]:
pd.DataFrame(columns=id_cols + monthly_cols + ['Month'])

,Plant Id,Plant State,NERC Region,AER Fuel Type Code,Quantity,Elec_Quantity,MMBtuPer_Unit,Tot_MMBtu,Elec_MMBtu,Netgen


In [113]:
gen_list = []
for month in months:
    gen_df = pd.DataFrame(columns=id_cols + monthly_cols)
    
    # Took me a few tries to figure out that I couldn't use .loc for gen_df
    gen_df[id_cols] = generation.loc[:,id_cols]
    gen_df['Month'] = month
    
    for col in monthly_cols:
        gen_df.loc[:,col] = generation.loc[:,col + ' ' + month]
    
    gen_list.append(gen_df)

In [120]:
gen_stack = pd.concat(gen_list)
gen_stack.describe()

,Plant Id,Quantity,Elec_Quantity,MMBtuPer_Unit,Tot_MMBtu,Elec_MMBtu,Netgen
count,82152.000000,8.215200e+04,8.215200e+04,82152.000000,8.215200e+04,8.215200e+04,8.215200e+04
mean,47395.247590,1.446661e+05,1.269590e+05,2.397057,4.205770e+05,3.947495e+05,4.181968e+04
std,39656.418102,6.144502e+05,5.480218e+05,5.409176,1.371208e+06,1.359613e+06,1.331186e+05
min,3.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,-1.164190e+05
25%,6001.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
50%,55061.000000,0.000000e+00,0.000000e+00,0.000000,2.546500e+03,1.822500e+03,3.217385e+02
75%,99999.000000,1.622225e+04,9.554750e+03,1.040000,1.742685e+05,1.271422e+05,1.720887e+04
max,99999.000000,1.650352e+07,1.356701e+07,34.180000,1.966522e+07,1.966522e+07,1.940054e+06


### Tag lines as using a combustion fuel or not

In [115]:
gen_stack['AER Fuel Type Code'].unique()

array([u'NG', u'COL', u'HYC', u'DFO', u'NUC', u'WOO', u'HPS', u'SUN',
       u'RFO', u'MLG', u'PC', u'ORW', u'GEO', u'OTH', u'OOG', u'WWW',
       u'WOC', nan, u'WND'], dtype=object)

In [116]:
non_combust = ['HYC', 'NUC', 'SUN', 'GEO', 'WND'] # might be incomplete

In [117]:
def tag_combust(row):
    if row['AER Fuel Type Code'] in non_combust:
        return 0
    else:
        return 1

In [121]:
gen_stack['Combust'] = gen_stack.apply(tag_combust, axis=1)

In [122]:
gen_stack.head()

,Plant Id,Plant State,NERC Region,AER Fuel Type Code,Quantity,Elec_Quantity,MMBtuPer_Unit,Tot_MMBtu,Elec_MMBtu,Netgen,Month,Combust
0,3,AL,SERC,NG,57253,57253,1.017,58226,58226,268797.000,January,1
1,3,AL,SERC,NG,5248798,5248798,1.017,5338028,5338028,511773.000,January,1
2,3,AL,SERC,COL,159951,159951,20.589,3293231,3293231,329513.400,January,1
3,3,AL,SERC,NG,129803,129803,1.016,131880,131880,13195.605,January,1
4,4,AL,SERC,HYC,0,0,0.000,1301314,1301314,139641.000,January,0


## Now group and sum
Only keep data for May

In [149]:
test = gen_stack.loc[gen_stack['Month']=='May',:].groupby(['Plant Id', 'NERC Region']).sum()
test.head()

,,Quantity,Elec_Quantity,MMBtuPer_Unit,Tot_MMBtu,Elec_MMBtu,Netgen,Combust
Plant Id,NERC Region,,,,,,,
3,SERC,4662196,4662196,24.476,8393587,8393587,1028473.000,4
4,SERC,0,0,0.000,95361,95361,10233.000,0
8,SERC,188233,188233,30.364,4588547,4588547,471836.004,2
10,SERC,116191,116191,2.066,120025,120025,6734.000,7
14,SERC,0,0,0.000,132060,132060,14171.000,0


In [150]:
test.reset_index('NERC Region')

,NERC Region,Quantity,Elec_Quantity,MMBtuPer_Unit,Tot_MMBtu,Elec_MMBtu,Netgen,Combust
Plant Id,,,,,,,,
3,SERC,4662196,4662196,24.476,8393587,8393587,1028473.000,4
4,SERC,0,0,0.000,95361,95361,10233.000,0
8,SERC,188233,188233,30.364,4588547,4588547,471836.004,2
10,SERC,116191,116191,2.066,120025,120025,6734.000,7
14,SERC,0,0,0.000,132060,132060,14171.000,0
15,SERC,0,0,0.000,219444,219444,23548.000,0
16,SERC,0,0,0.000,145265,145265,15588.000,0
17,SERC,0,0,0.000,193742,193742,20790.000,0
18,SERC,0,0,0.000,54386,54386,5836.000,0


In [123]:
facility_gen = gen_stack.loc[gen_stack['Month']=='May',:].groupby('Plant Id').sum()

In [124]:
facility_gen.head()

,Quantity,Elec_Quantity,MMBtuPer_Unit,Tot_MMBtu,Elec_MMBtu,Netgen,Combust
Plant Id,,,,,,,
3,4662196,4662196,24.476,8393587,8393587,1028473.000,4
4,0,0,0.000,95361,95361,10233.000,0
8,188233,188233,30.364,4588547,4588547,471836.004,2
10,116191,116191,2.066,120025,120025,6734.000,7
14,0,0,0.000,132060,132060,14171.000,0


If I want to keep the NERC Region, I can do that in the groupby

In [153]:
facility_gen = gen_stack.loc[gen_stack['Month']=='May',:].groupby(['Plant Id', 'NERC Region']).sum()
facility_gen.head()

,,Quantity,Elec_Quantity,MMBtuPer_Unit,Tot_MMBtu,Elec_MMBtu,Netgen,Combust
Plant Id,NERC Region,,,,,,,
3,SERC,4662196,4662196,24.476,8393587,8393587,1028473.000,4
4,SERC,0,0,0.000,95361,95361,10233.000,0
8,SERC,188233,188233,30.364,4588547,4588547,471836.004,2
10,SERC,116191,116191,2.066,120025,120025,6734.000,7
14,SERC,0,0,0.000,132060,132060,14171.000,0


In [154]:
facility_gen.reset_index('NERC Region', inplace=True)
facility_gen.head()

,NERC Region,Quantity,Elec_Quantity,MMBtuPer_Unit,Tot_MMBtu,Elec_MMBtu,Netgen,Combust
Plant Id,,,,,,,,
3,SERC,4662196,4662196,24.476,8393587,8393587,1028473.000,4
4,SERC,0,0,0.000,95361,95361,10233.000,0
8,SERC,188233,188233,30.364,4588547,4588547,471836.004,2
10,SERC,116191,116191,2.066,120025,120025,6734.000,7
14,SERC,0,0,0.000,132060,132060,14171.000,0


# Merge data from all three sources

In [155]:
merged = facility_gen.merge(facility_cap, how='inner', left_index=True, right_index=True)

In [156]:
merged.describe()

,Quantity,Elec_Quantity,MMBtuPer_Unit,Tot_MMBtu,Elec_MMBtu,Netgen,Combust,Nameplate Capacity (MW)
count,2.204000e+03,2.204000e+03,2204.000000,2.204000e+03,2.204000e+03,2.204000e+03,2204.000000,2204.000000
mean,4.341980e+05,3.940222e+05,6.088443,1.324327e+06,1.269274e+06,1.349217e+05,1.699183,484.658122
std,1.116450e+06,1.004303e+06,12.526411,2.633492e+06,2.631382e+06,2.663379e+05,1.944481,615.046700
min,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,-6.372300e+04,0.000000,1.800000
25%,0.000000e+00,0.000000e+00,0.000000,1.038605e+05,9.849125e+04,1.000100e+04,0.000000,94.575000
50%,3.765000e+02,2.850000e+02,0.924500,3.455885e+05,3.064225e+05,3.220900e+04,1.000000,233.900000
75%,2.545605e+05,2.182568e+05,3.066000,1.264416e+06,1.035504e+06,1.138057e+05,3.000000,650.000000
max,1.622117e+07,1.124778e+07,99.754000,2.580555e+07,2.580555e+07,2.467542e+06,11.000000,6809.000000


Save the non-combustion units, because I'm going to join the merged dataframe with the epa dataframe and want to add back in the non-combustion

In [157]:
non_combust = merged.loc[merged['Combust']==0,:]
non_combust.describe()

,Quantity,Elec_Quantity,MMBtuPer_Unit,Tot_MMBtu,Elec_MMBtu,Netgen,Combust,Nameplate Capacity (MW)
count,933.0,933.0,933.0,9.330000e+02,9.330000e+02,9.330000e+02,933.0,933.000000
mean,0.0,0.0,0.0,1.090682e+06,1.090682e+06,1.088466e+05,0.0,248.966131
std,0.0,0.0,0.0,3.094933e+06,3.094933e+06,2.976431e+05,0.0,463.451807
min,0.0,0.0,0.0,0.000000e+00,0.000000e+00,-1.500000e+01,0.0,1.800000
25%,0.0,0.0,0.0,1.239710e+05,1.239710e+05,1.330300e+04,0.0,61.200000
50%,0.0,0.0,0.0,2.510540e+05,2.510540e+05,2.694000e+04,0.0,110.000000
75%,0.0,0.0,0.0,4.913350e+05,4.913350e+05,5.272400e+04,0.0,200.000000
max,0.0,0.0,0.0,2.580555e+07,2.580555e+07,2.467542e+06,0.0,4209.600000


In [158]:
non_combust.head()

,NERC Region,Quantity,Elec_Quantity,MMBtuPer_Unit,Tot_MMBtu,Elec_MMBtu,Netgen,Combust,Nameplate Capacity (MW)
4,SERC,0,0,0.0,95361,95361,10233.0,0,225.0
14,SERC,0,0,0.0,132060,132060,14171.0,0,128.1
15,SERC,0,0,0.0,219444,219444,23548.0,0,177.0
16,SERC,0,0,0.0,145265,145265,15588.0,0,210.6
17,SERC,0,0,0.0,193742,193742,20790.0,0,170.0


In [159]:
merged = merged.merge(facility_emiss, how='inner', left_index=True, right_index=True)
merged.describe()

,Quantity,Elec_Quantity,MMBtuPer_Unit,Tot_MMBtu,Elec_MMBtu,Netgen,Combust,Nameplate Capacity (MW),Operating Time,Gross Load (MW-h),Steam Load (1000lb),SO2 (tons),Avg. NOx Rate (lb/MMBtu),NOx (tons),CO2 (short tons),Heat Input (MMBtu),EPA Region
count,9.020000e+02,9.020000e+02,902.000000,9.020000e+02,9.020000e+02,9.020000e+02,902.000000,902.000000,902.000000,7.870000e+02,2.800000e+01,804.000000,810.000000,810.000000,7.820000e+02,8.100000e+02,902.000000
mean,8.026699e+05,7.984092e+05,9.428446,1.737592e+06,1.732175e+06,1.880601e+05,2.871397,811.238581,825.084268,2.198615e+05,4.814780e+05,107.486081,0.493989,92.464405,1.687089e+05,1.937353e+06,5.252772
std,1.368488e+06,1.361816e+06,12.991945,2.442123e+06,2.440015e+06,2.605284e+05,1.620855,651.587349,892.628864,2.760750e+05,5.343461e+05,300.205896,1.316566,187.144645,2.448701e+05,2.477358e+06,2.177569
min,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,-5.056000e+03,1.000000,27.000000,0.000000,0.000000e+00,5.103100e+02,0.000000,-7.815200,0.000000,0.000000e+00,3.400000e+00,1.000000
25%,3.070300e+04,3.037650e+04,1.031250,6.053450e+04,6.040225e+04,4.117250e+03,2.000000,349.050000,89.235000,1.383868e+04,7.159151e+04,0.086750,0.059175,3.705000,1.273775e+04,1.515684e+05,4.000000
50%,2.210980e+05,2.210980e+05,2.069500,7.232245e+05,7.149535e+05,7.754000e+04,2.000000,631.000000,579.315000,1.152687e+05,3.027295e+05,0.557500,0.179250,12.956000,8.169769e+04,1.027198e+06,5.000000
75%,9.083002e+05,9.083002e+05,18.052750,2.485223e+06,2.470881e+06,2.947978e+05,4.000000,1071.050000,1326.427500,3.320322e+05,6.532815e+05,44.930750,0.435050,73.797000,2.095907e+05,2.694102e+06,7.000000
max,1.124778e+07,1.124778e+07,75.589000,1.681146e+07,1.681146e+07,1.775310e+06,11.000000,4317.500000,5190.810000,1.652769e+06,1.783969e+06,3639.271000,17.099800,1722.928000,1.607836e+06,1.567091e+07,10.000000


In [160]:
merged.head()

,NERC Region,Quantity,Elec_Quantity,MMBtuPer_Unit,Tot_MMBtu,Elec_MMBtu,Netgen,Combust,Nameplate Capacity (MW),Operating Time,Gross Load (MW-h),Steam Load (1000lb),SO2 (tons),Avg. NOx Rate (lb/MMBtu),NOx (tons),CO2 (short tons),Heat Input (MMBtu),EPA Region
3,SERC,4662196,4662196,24.476,8393587,8393587,1028473.000,4,2569.5,3844.00,1048450.25,NaN,595.911,0.5760,411.165,677078.976,8471790.125,4.0
8,SERC,188233,188233,30.364,4588547,4588547,471836.004,2,1166.7,1837.25,505777.25,NaN,125.662,0.5375,440.348,531418.603,5179513.525,4.0
10,SERC,116191,116191,2.066,120025,120025,6734.000,7,1288.4,188.25,8897.75,NaN,48.579,0.8474,10.162,11340.124,144258.150,4.0
26,SERC,197510,197510,30.196,4577634,4577634,453833.000,4,2034.0,744.00,495791.00,NaN,173.698,0.0646,165.370,524604.900,5113099.400,4.0
47,SERC,58,58,5.800,336,336,62.000,7,1026.0,20.00,177.00,NaN,0.333,4.4379,0.722,192.400,2765.600,4.0


Now concat the two dataframes

In [161]:
final = pd.concat([merged, non_combust])
final

,Avg. NOx Rate (lb/MMBtu),CO2 (short tons),Combust,EPA Region,Elec_MMBtu,Elec_Quantity,Gross Load (MW-h),Heat Input (MMBtu),MMBtuPer_Unit,NERC Region,NOx (tons),Nameplate Capacity (MW),Netgen,Operating Time,Quantity,SO2 (tons),Steam Load (1000lb),Tot_MMBtu
3,0.5760,677078.976,4,4.0,8393587,4662196,1048450.25,8471790.125,24.476,SERC,411.165,2569.5,1028473.000,3844.00,4662196,595.911,NaN,8393587
8,0.5375,531418.603,2,4.0,4588547,188233,505777.25,5179513.525,30.364,SERC,440.348,1166.7,471836.004,1837.25,188233,125.662,NaN,4588547
10,0.8474,11340.124,7,4.0,120025,116191,8897.75,144258.150,2.066,SERC,10.162,1288.4,6734.000,188.25,116191,48.579,NaN,120025
26,0.0646,524604.900,4,4.0,4577634,197510,495791.00,5113099.400,30.196,SERC,165.370,2034.0,453833.000,744.00,197510,173.698,NaN,4577634
47,4.4379,192.400,7,4.0,336,58,177.00,2765.600,5.800,SERC,0.722,1026.0,62.000,20.00,58,0.333,NaN,336
50,NaN,NaN,2,4.0,0,0,NaN,NaN,0.000,SERC,NaN,575.0,0.000,0.00,0,NaN,NaN,0
51,0.2080,270128.775,3,6.0,2799148,241664,262392.63,2481213.560,14.902,SPP,241.863,720.7,238020.998,553.48,241664,1081.656,NaN,2799148
54,0.3753,11101.898,2,4.0,189569,177832,15652.65,186799.693,1.066,SERC,2.787,1055.0,15037.000,254.45,177832,0.057,NaN,189569
56,0.5125,129301.129,2,4.0,1098557,49159,112793.47,1260257.488,28.560,SERC,167.878,538.0,97056.000,759.23,49159,63.902,NaN,1098557
60,0.2169,44877.292,2,7.0,404792,24713,36111.00,427905.242,22.206,MRO,39.165,324.3,31448.000,753.95,24713,176.503,NaN,404792


In [162]:
final.index.rename('Plant ID', inplace=True)

In [163]:
final.describe()

,Avg. NOx Rate (lb/MMBtu),CO2 (short tons),Combust,EPA Region,Elec_MMBtu,Elec_Quantity,Gross Load (MW-h),Heat Input (MMBtu),MMBtuPer_Unit,NOx (tons),Nameplate Capacity (MW),Netgen,Operating Time,Quantity,SO2 (tons),Steam Load (1000lb),Tot_MMBtu
count,810.000000,7.820000e+02,1835.000000,902.000000,1.835000e+03,1.835000e+03,7.870000e+02,8.100000e+02,1835.000000,810.000000,1835.000000,1.835000e+03,902.000000,1.835000e+03,804.000000,2.800000e+01,1.835000e+03
mean,0.493989,1.687089e+05,1.411444,5.252772,1.406010e+06,3.924605e+05,2.198615e+05,1.937353e+06,4.634582,92.464405,525.352916,1.477843e+05,825.084268,3.945549e+05,107.486081,4.814780e+05,1.408673e+06
std,1.316566,2.448701e+05,1.830965,2.177569,2.809884e+06,1.034649e+06,2.760750e+05,2.477358e+06,10.254386,187.144645,629.910855,2.827274e+05,892.628864,1.039785e+06,300.205896,5.343461e+05,2.811094e+06
min,-7.815200,0.000000e+00,0.000000,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,3.400000e+00,0.000000,0.000000,1.800000,-5.056000e+03,0.000000,0.000000e+00,0.000000,5.103100e+02,0.000000e+00
25%,0.059175,1.273775e+04,0.000000,4.000000,1.052255e+05,0.000000e+00,1.383868e+04,1.515684e+05,0.000000,3.705000,102.000000,1.072450e+04,89.235000,0.000000e+00,0.086750,7.159151e+04,1.056730e+05
50%,0.179250,8.169769e+04,0.000000,5.000000,3.320640e+05,0.000000e+00,1.152687e+05,1.027198e+06,0.000000,12.956000,257.000000,3.389400e+04,579.315000,0.000000e+00,0.557500,3.027295e+05,3.320640e+05
75%,0.435050,2.095907e+05,2.000000,7.000000,1.256538e+06,2.050515e+05,3.320322e+05,2.694102e+06,2.064000,73.797000,696.150000,1.284585e+05,1326.427500,2.050515e+05,44.930750,6.532815e+05,1.256538e+06
max,17.099800,1.607836e+06,11.000000,10.000000,2.580555e+07,1.124778e+07,1.652769e+06,1.567091e+07,75.589000,1722.928000,4317.500000,2.467542e+06,5190.810000,1.124778e+07,3639.271000,1.783969e+06,2.580555e+07


In [164]:
final['CO2 (short tons)'].sum() * 2000 * 2.2046 / final['Netgen'].sum()

2145.06392182917